In [ ]:
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage, TextContentItem, ImageContentItem, ImageUrl

from azure.core.credentials import AzureKeyCredential

In [ ]:
# For Serverless API or Managed Compute endpoints
client = ChatCompletionsClient(
    # endpoint="https://DeepSeek-R1-arena.westus3.models.ai.azure.com",
    # endpoint="https://Phi-4-arena.westus3.models.ai.azure.com",
    # credential=AzureKeyCredential("ZI5fQw13K3a148fQs1KU6dU2ja0GAAf8")
    # NVIDIA-NIMS models test:
    endpoint="https://cpu-dt-test.westus3.inference.ml.azure.com/",
    credential=AzureKeyCredential(
        ""
    ),
    headers={
        "azureml-model-deployment": "proteinmpnn-model-custom-1234",
    },
)

# # "The auth token does not have access to the endpoint. Please grant Azure AI Developer role"
# dft_client = ChatCompletionsClient(
#     endpoint="https://DeepSeek-R1-arena.westus3.models.ai.azure.com/chat/completions",
#     credential=DefaultAzureCredential()
# )

In [ ]:
from urllib.request import urlopen, Request
import base64

image_url = "https://news.microsoft.com/source/wp-content/uploads/2024/04/The-Phi-3-small-language-models-with-big-potential-1-1900x1069.jpg"
image_format = "jpeg"

request = Request(image_url, headers={"User-Agent": "Mozilla/5.0"})
image_data = base64.b64encode(urlopen(request).read()).decode("utf-8")
data_url = f"data:image/{image_format};base64,{image_data}"

In [ ]:
import requests
import IPython.display as Disp

Disp.Image(requests.get(image_url).content)

In [ ]:
response = client.complete(
    messages=[
        SystemMessage(content="You are who you want to be."),
        UserMessage(content=[
            TextContentItem(text="What is the capital of France?"),
            # ImageContentItem(image_url=ImageUrl(url=data_url))
        ])
    ],
    max_tokens=2048,
    # model="DeepSeek-R1-arena"
)

print(response.choices[0].message.content)

___

In [ ]:
import requests
import os
import json
from pathlib import Path

def get_reduced_pdb():
    pdb = Path("../data/1R42.pdb")
    if not pdb.exists():
        pdb.write_text(requests.get("https://files.rcsb.org/download/1R42.pdb").text)
    lines = filter(lambda line: line.startswith("ATOM"), pdb.read_text().split("\n"))
    return "\n".join(list(lines)[:200])


In [ ]:
endpoint = "https://cpu-dt-test.westus3.inference.ml.azure.com/biology/ipd/proteinmpnn/predict"
api_key = ""
deployment_name = "proteinmpnn-model-custom-1234"

data = {
    "input_pdb": get_reduced_pdb(),
    "ca_only": False,
    "use_soluble_model": False,
    "sampling_temp": [0.1]
}

In [ ]:
endpoint = "https://cpu-dt-test.westus3.inference.ml.azure.com/v1/vista3d/inference"
# endpoint = "https://cpu-dt-test.westus3.inference.ml.azure.com/v1/medicalimaging/nvidia/vista-3d"
api_key = ""
deployment_name = "vista3d-model-custom-2"

sample = "example-1"
data = {
    "image": f"https://assets.ngc.nvidia.com/products/api-catalog/vista3d/{sample}.nii.gz",
    "prompts": {
        "classes": ["liver", "spleen"]
    }
}

In [ ]:
import io
import os
import requests
import shutil
import tempfile
import zipfile
invoke_url = "https://cpu-dt-test.westus3.inference.ml.azure.com/v1/vista3d/inference"
headers = {
    "Authorization": f"Bearer {api_key}",
}
sample = "example-1"
payload = {
    "image": f"https://assets.ngc.nvidia.com/products/api-catalog/vista3d/{sample}.nii.gz",
    "prompts": {
        "classes": ["liver", "spleen"]
    }
}
# re-use connections
session = requests.Session()
response = session.post(invoke_url, headers=headers, json=payload)
print(response.headers)
response.raise_for_status()
with tempfile.TemporaryDirectory() as temp_dir:
    z = zipfile.ZipFile(io.BytesIO(response.content))
    z.extractall(temp_dir)
    file_list = os.listdir(temp_dir)
    for filename in file_list:
        filepath = os.path.join(temp_dir, filename)
        if os.path.isfile(filepath) and filename.endswith(".nrrd"):
            print(f"Downloaded {filename}")
print("---------------------------------------------------------------")
print(f"Input Image: {payload['image']}")
print(f"Class Prompts: {payload.get('prompts', {}).get('classes')}")
print(f"Response mask: {sample}_seg.nrrd")
print("")

In [ ]:
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}",
    "azureml-model-deployment": deployment_name,
}

In [ ]:
import io
import os
import requests
import shutil
import tempfile
import zipfile

sample = "example-1"
payload = {
    "image": f"https://assets.ngc.nvidia.com/products/api-catalog/vista3d/{sample}.nii.gz",
    "prompts": {
        "classes": ["liver", "spleen"]
    }
}

# re-use connections
session = requests.Session()
response = session.post(endpoint, headers=headers, json=data)

response.raise_for_status()
with tempfile.TemporaryDirectory() as temp_dir:
    z = zipfile.ZipFile(io.BytesIO(response.content))
    z.extractall(temp_dir)
    file_list = os.listdir(temp_dir)
    for filename in file_list:
        filepath = os.path.join(temp_dir, filename)
        if os.path.isfile(filepath) and filename.endswith(".nrrd"):
            print(f"Downloaded {filename}")

print("---------------------------------------------------------------")
print(f"Input Image: {payload['image']}")
print(f"Class Prompts: {payload.get('prompts', {}).get('classes')}")
print(f"Response mask: {sample}_seg.nrrd")
print("")

In [ ]:
import urllib.request
import json

body = str.encode(json.dumps(data))


client = requests.Session()
client.headers.update(headers)

response = client.post(endpoint, json=data)
print(response.headers)
response.raise_for_status()
result = response.json()
print("Result:")
print(result)


In [ ]:
# get request to https://cpu-dt-test.westus3.inference.ml.azure.com/swagger.json
response = client.get("https://cpu-dt-test.westus3.inference.ml.azure.com/v1/health/ready")
# response = client.get(endpoint)
response.raise_for_status()
print(response.json())